<a href="https://colab.research.google.com/github/amoyag/GNN/blob/main/GNN_BS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A GNN toy model
A Graph Neural Network (GNN) model that's educational and suitable for a systems biology workshop. It's designed to analyze biological networks like protein-protein interactions or metabolic pathways. It uses PyTorch Geometric:

1. The `BiologyGNN` class is a simple GNN with:
   - 3 graph convolutional layers
   - ReLU activation functions
   - Dropout for regularization
   - A final linear layer for classification

2. Helper functions include:
   - `create_example_dataset()`: Creates a sample biological network
   - `train_model()`: Handles the training loop



In [ ]:
# check PyTorch and CUDA versions

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

In [ ]:

# use those exact versions in the installation
!pip install torch-geometric pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.5.1+cu121.html

# check the installation
import torch_geometric
print(f"PyG version: {torch_geometric.__version__}")

In [ ]:
# install other stuff
!pip install  bravado pandas numpy networkx

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import networkx as nx
import numpy as np

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Add this at the start of your main code
set_seed(42)  # or any other number

class BiologyGNN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes, hidden_channels=64):
        super(BiologyGNN, self).__init__()

        # Graph Convolutional layers
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)

        # Output layer
        self.linear = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        # First Graph Convolution Layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.1, training=self.training)

        # Second Graph Convolution Layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.1, training=self.training)

        # Third Graph Convolution Layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)

        # Output Layer
        x = self.linear(x)

        return F.log_softmax(x, dim=1)

def create_example_dataset():
    """Create a simple example biological network dataset"""
    # Create a random graph
    G = nx.random_geometric_graph(20, 0.3)

    # Convert to edge index format
    edge_index = torch.tensor([[e[0] for e in G.edges()],
                             [e[1] for e in G.edges()]], dtype=torch.long)

    # Create random node features (could represent gene expression, protein properties, etc.)
    num_node_features = 10
    x = torch.randn((20, num_node_features))

    # Create random node labels (could represent protein function, pathway membership, etc.)
    y = torch.randint(0, 3, (20,))

    # Create PyG Data object
    data = Data(x=x, edge_index=edge_index, y=y)
    return data

def train_model(model, data, epochs=100):
    """Train the GNN model"""
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1:03d}, Loss: {loss:.4f}')



#### Evaluate the model



def evaluate_model(model, data):
    """Evaluate model performance with multiple metrics"""
    model.eval()
    with torch.no_grad():
        # Get predictions
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        true = data.y

        # Calculate metrics
        accuracy = accuracy_score(true, pred) * 100
        precision, recall, f1, _ = precision_recall_fscore_support(true, pred, average=None)
        conf_matrix = confusion_matrix(true, pred)

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted Class')
        plt.ylabel('True Class')
        plt.show()

        # Print results
        print(f"\nModel Performance:")
        print(f"Overall Accuracy: {accuracy:.1f}%")
        for i in range(len(precision)):
            print(f"\nClass {i}:")
            print(f"Precision: {precision[i]*100:.1f}%")
            print(f"Recall: {recall[i]*100:.1f}%")
            print(f"F1 Score: {f1[i]*100:.1f}%")








In [ ]:

# Example usage
if __name__ == "__main__":
    # Create example dataset
    data = create_example_dataset()

    # Initialize model
    model = BiologyGNN(num_node_features=10, num_classes=3)

    # Train model
    train_model(model, data)

    # Make predictions
    model.eval()
    pred = model(data.x, data.edge_index)
    predicted_classes = pred.argmax(dim=1)
    print("\nPredicted classes:", predicted_classes)

### Break down of the results:


1. The Loss Values (Epoch lines):
  - These show how well the model is learning over time (epochs). Loss starts high and decreases, which means means better performance. It shows the model is learning successfully

  Loss is a measure of how wrong the model's predictions are. Think of it as the "error score" - the lower the loss, the better the model is performing.

  The code
  ```python
  loss = F.nll_loss(out, data.y)
```
  tells the system to: make predictions (`out`). We compare these predictions to the true labels (`data.y`). The loss function calculates how far off our predictions are This loss value guides the model in adjusting its parameters to make better predictions.

  
  Think of training a neural network like teaching someone to play a pattern recognition game. In our case, the model is trying to guess which category (0, 1, or 2) each node in the network belongs to. Every time it makes a guess, it gets feedback in the form of a loss value —essentially a score of how many mistakes it made. The model then uses this feedback to adjust its strategy and make better guesses next time, with a lower loss value indicating fewer mistakes.

In the context of biological networks, this becomes particularly meaningful. For instance, the model might be trying to predict whether specific proteins are involved in cancer (class 0) or not (class 1). The loss value would indicate how accurate these predictions are, with a theoretical perfect score of 0 meaning the model correctly identified every protein's role. However, achieving such perfect prediction is rarely possible in practice due to the complexity of biological systems and the inherent noise in biological data.

2. The Predicted Classes:
  ```
  tensor([0, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 1])
  ```
  This shows the model's predictions for each of the 20 nodes (proteins/genes) in your example network, where:
  - Each number (0, 1, or 2) represents a different class
  - In this example, these are random classes since we used `create_example_dataset()`
  - In a real biological context, these could represent:
    - Protein functions
    - Pathway membership
    - Disease association
    - Any other categorical property you're trying to predict

  However, it's important to note that this is using synthetic data where:
  - The network structure is random (from `nx.random_geometric_graph`)
  - The node features are random (from `torch.randn`)
  - The true labels are random (from `torch.randint`)

### About the model and the data
The Graph Neural Network (GNN) used here is designed to learn patterns from network-structured biological data. The model architecture consists of three graph convolutional layers (GCNConv) followed by a final linear layer. Each convolutional layer learns to aggregate information from neighboring nodes, meaning each protein learns from the proteins it interacts with. After each convolution, the model applies a ReLU activation function and dropout (with probability 0.1) to prevent overfitting. The final linear layer maps the learned representations to class probabilities using softmax, effectively making predictions about which category each protein belongs to.

The data structure represents a simplified biological network with 20 nodes (proteins) connected randomly using NetworkX's geometric graph generator. Each protein in this network has 10 random features (data.x), which in a real biological context could represent properties like expression levels, molecular weight, or sequence characteristics. The connections between proteins (edges) are established with a proximity threshold of 0.3, meaning proteins are connected if they're within this geometric distance of each other. Each protein is also randomly assigned one of three classes (0, 1, or 2; data.y), simulating a classification task like predicting protein function or disease association. While this synthetic data helps demonstrate how the model works, in a real application you'd replace these random elements with actual protein interaction data, meaningful biological features, and known functional annotations.

### About training the GNN

Think of the ground truth classes (data.y) as labels we already know, like having a set of proteins where we know their functions (some are enzymes, some are transporters, etc.). The model doesn't use these labels to make its predictions - instead, it uses two other pieces of information: the protein features (data.x) and who interacts with whom (edge_index).

When making a prediction, the model looks at each protein and considers:
1. The protein's own features (like its molecular weight, sequence properties, etc.)
2. The features of proteins it's connected to in the network (its interaction partners)

Through the training process, the model learns patterns like "proteins with these features that interact with these types of partners tend to be class 1." It's similar to how biologists might infer a protein's function by looking at its properties and what other proteins it interacts with. The model is essentially trying to rediscover the relationship between these patterns and the known classes.

Then we compare the model's predictions to those ground truth labels to see how well it learned these patterns. This is where the loss comes in —it measures how different the predictions are from the real classes we knew all along.

In [ ]:
#@title Model evaluation
#Use this with your existing model and data
evaluate_model(model, data)

### About the evaluation parmeters


**Accuracy** is the most straightforward metric - it's simply the percentage of proteins that the model classified correctly. However, accuracy alone can be misleading, especially if the classes are imbalanced (like having many more proteins of one class than others).

**Precision** tells us, when the model predicts a specific class, how often it's correct. For example, if the model has 80% precision for class 0, it means that when it predicts a protein belongs to class 0, it's right 80% of the time. This is particularly important when false positives are costly.

**Recall** (also called sensitivity) shows how many proteins of a particular class the model actually found. If the recall for class 1 is 70%, it means the model correctly identified 70% of all proteins that truly belong to class 1. This matters when missing cases would be problematic.

The **F1 score** balances precision and recall into a single number. It's particularly useful when you need to find a sweet spot between not missing true cases (recall) and not making false predictions (precision). An F1 score of 100% would mean perfect precision and recall.

The confusion matrix provides a complete picture of the model's predictions. Each row represents the actual class, and each column shows what the model predicted. The numbers on the diagonal show correct predictions, while off-diagonal numbers show mistakes. This helps identify which classes the model commonly confuses with each other.

### Deep dive on the data


We're working with a network of 20 proteins, where each protein has 10 numerical features associated with it. These features are currently random numbers that simulate real protein properties - in a real biological context, they might represent characteristics like molecular weight, sequence patterns, or expression levels.

The network's structure is defined by connections between proteins, which we call edges. Each edge represents a potential interaction between two proteins, created using a geometric proximity rule - proteins that are "closer" to each other in our simulated space are connected. This gives us a pattern of connections that mimics real protein interaction networks, though in a simplified way. We can visualize this network as a graph where each protein is a node, and lines between nodes show their interactions.

Each protein in our network is also assigned to one of three classes (0, 1, or 2). These class assignments are currently random, but in a real biological context, they might represent functional categories, like whether a protein is an enzyme or a transcription factor. The distribution of these classes tells us how many proteins belong to each category.

To understand the network's overall structure, we can look at metrics like the average number of interactions per protein (average degree) and network density, which tells us how many of the possible connections between proteins actually exist. These properties help us understand how tightly connected our protein network is. The network visualization shows us this structure spatially, with different colors representing the different classes, giving us a visual sense of how proteins of different types are distributed through the network.

The GNN learns by combining and processing two key types of information for each protein: its own **features** and the features of proteins it **interacts** with. During each pass through the network (using the three **graph convolutional layers**), a protein's **representation** is updated by **aggregating information** from its neighbors. Think of it like each protein getting a better understanding of its role not just from its own characteristics, but also from the company it keeps in the interaction network. Each layer of the GNN allows this information to spread further through the network, so proteins eventually gain information from both **direct interaction partners** and more **distant connections**.

The model's **learning process** involves finding patterns between these combined feature representations and the **class assignments**. For example, it might learn that proteins with certain feature values that interact with proteins having complementary features tend to belong to a particular class. The **weights** in the convolutional layers are adjusted through training to better capture these patterns. When the model makes a **prediction** for a protein, it's essentially saying "based on this protein's features and the features of its interaction partners, it shows patterns similar to other proteins of class X that I've seen during training." The effectiveness of this learning process depends heavily on how **informative** the features are and how **meaningful** the network structure is, which is why using real biological data with actual protein properties and verified interactions would give more meaningful results than our random example.

In [ ]:

def examine_network_data(data):
    """Examine and display network data features"""
    # Print basic information
    print(f"Number of nodes (proteins): {data.x.shape[0]}")
    print(f"Number of features per node: {data.x.shape[1]}")
    print(f"Number of edges (interactions): {data.edge_index.shape[1]}")
    print(f"\nFeature matrix shape: {data.x.shape}")

    # Show example of features for first few nodes
    print("\nExample features for first 3 proteins:")
    for i in range(3):
        print(f"Protein {i} features: {data.x[i].numpy()}")

    # Show class distribution
    unique_classes, counts = torch.unique(data.y, return_counts=True)
    print("\nClass distribution:")
    for class_idx, count in zip(unique_classes, counts):
        print(f"Class {class_idx}: {count} proteins")

    # Visualize network
    G = nx.Graph()
    edge_list = data.edge_index.t().numpy()
    for edge in edge_list:
        G.add_edge(edge[0], edge[1])

    plt.figure(figsize=(10, 10))
    pos = nx.spring_layout(G)
    nx.draw(G, pos,
            node_color=data.y,  # Color nodes by their class
            node_size=500,
            cmap=plt.cm.viridis)
    plt.title("Network Structure (colors represent classes)")
    plt.show()

    return G



In [ ]:
# examine the  dataset
#data = create_example_dataset()
G = examine_network_data(data)

# Additional network analysis
avg_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()
print(f"\nAverage number of interactions per protein: {avg_degree:.2f}")
print(f"Network density: {nx.density(G):.3f}")

### Going a step further

In a real-world scenario, we often face situations where not all proteins in a network have known functions or classifications. This is where Graph Neural Networks become particularly valuable. Rather than requiring complete class information for all proteins, GNNs can learn from partially labeled networks where only some proteins have known classes. The model learns patterns from the proteins with known classifications and uses these patterns, along with protein features and network structure, to predict the classes of unlabeled proteins.

For example, in a network of 20 proteins, we might only know the functional classes of 17 proteins. The GNN can learn from these 17 proteins and predict the functions of the remaining 3 based on their features and their interactions with proteins of known function. This capability is especially valuable in biology since experimentally determining protein functions is time-consuming and expensive, while interaction data and protein features are often more readily available. The model essentially leverages what we already know about some proteins to make educated predictions about others.





The `create_partially_labeled_dataset()` function creates our test network with some proteins having unknown classes. It uses `random_geometric_graph` to generate the network structure and creates `num_features` random features for each protein. The key difference is that it marks some proteins as unknown by setting their class labels to `-1`. It selects which proteins to leave unlabeled using `torch.randperm` to randomly pick `num_unlabeled` proteins. Like our original dataset creation, it packages everything into a `Data` object containing the features (`x`), connections (`edge_index`), and labels (`y`).

The `train_with_partial_labels()` function modifies the training process to handle unlabeled proteins. It creates a `labeled_mask` to identify which proteins have known classes, and only uses these proteins for calculating the `loss`. The function still uses an `Adam` optimizer, but now it only compares the model's predictions against the known class labels, ignoring the proteins marked with `-1`. This lets the model learn from the labeled proteins while skipping the unlabeled ones during training.

The `predict_unknown_classes()` function takes our trained model and uses it to predict classes for the unlabeled proteins. It puts the model in evaluation mode with `model.eval()` and uses `torch.no_grad()` since we're not training anymore. It generates predictions for all proteins using the model, then uses an `unlabeled_mask` to find proteins with `-1` labels. For these proteins, it reports what class the model predicts based on their features and their connections to labeled proteins in the network.

In [ ]:
def create_partially_labeled_dataset(num_nodes=20, num_unlabeled=3):
    """Create a dataset where some proteins have unknown classes"""
    # Create basic dataset
    G = nx.random_geometric_graph(num_nodes, 0.3)
    edge_index = torch.tensor([[e[0] for e in G.edges()],
                             [e[1] for e in G.edges()]], dtype=torch.long)

    # Create features
    num_features = 10
    x = torch.randn((num_nodes, num_features))

    # Create labels, marking some as unknown (-1)
    y = torch.randint(0, 3, (num_nodes,))

    # Randomly select nodes to be unlabeled
    unlabeled_indices = torch.randperm(num_nodes)[:num_unlabeled]
    y[unlabeled_indices] = -1  # -1 indicates unknown class

    data = Data(x=x, edge_index=edge_index, y=y)
    return data

def train_with_partial_labels(model, data):
    """Train model using only labeled nodes"""
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Get mask for labeled nodes
    labeled_mask = data.y != -1

    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)

        # Calculate loss only on labeled nodes
        loss = F.nll_loss(out[labeled_mask], data.y[labeled_mask])
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1:03d}, Loss: {loss:.4f}')

def predict_unknown_classes(model, data):
    """Predict classes for unlabeled nodes"""
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)

        # Show predictions for unlabeled nodes
        unlabeled_mask = data.y == -1
        print("\nPredictions for unlabeled proteins:")
        for i in range(len(data.y)):
            if unlabeled_mask[i]:
                print(f"Protein {i}: Predicted Class {pred[i].item()}")

In [ ]:
# reate the partially labeled dataset
data = create_partially_labeled_dataset(num_nodes=20, num_unlabeled=3)

# Initialize and train the model
# use the prevoius model
# model = BiologyGNN(num_node_features=10, num_classes=3)
train_with_partial_labels(model, data)

# Get predictions for unlabeled proteins
predict_unknown_classes(model, data)

## Adding Expression Features to the Network: TCGA Data Integration
In order to enhance our protein-protein interaction network with meaningful biological features, we need to obtain gene expression data for each protein node. Gene expression levels from The Cancer Genome Atlas (TCGA) breast cancer dataset provide quantitative measurements of gene activity in tumor samples, which can serve as informative node features for our network. These expression values capture the abundance of each gene's transcripts, offering insights into their activity levels in the cancer context. By incorporating this high-dimensional molecular data as node features, we can generate richer embeddings that capture both the network topology and the underlying biological state of each protein, potentially improving our ability to identify cancer-related proteins through graph neural network analysis.


#### Explore data from cBioPortal
This code connects to cBioPortal and explores what data we can access through its API. It first lists all available methods specifically for molecular data, which we'll need for getting expression values. Then it shows all available resources in cBioPortal, giving us a complete view of what types of cancer genomics data we could potentially use.

In [ ]:
from bravado.client import SwaggerClient
import pandas as pd
import numpy as np

# Initialize cBioPortal client
cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/v2/api-docs',
    config={"validate_requests":False,"validate_responses":False,"validate_swagger_spec": False})

# Print available methods for Molecular_Data
print("Available methods for Molecular_Data:")
for method in dir(cbioportal.Molecular_Data):
    if not method.startswith('_'):
        print(method)

# Let's also check what other resources are available
print("\nAvailable resources:")
for resource in dir(cbioportal):
    if not resource.startswith('_'):
        print(resource)

#### Connect to cBioportal
This code sets up our connection to cBioPortal's API and examines what we need to provide to fetch expression data successfully. The first part imports required libraries and initializes the API client, while the second part inspects the parameters required by the API endpoint we'll use to get molecular data.

By checking these required parameters upfront, we ensure we understand exactly what information cBioPortal needs before we start fetching expression values for our network's proteins.

In [ ]:
from bravado.client import SwaggerClient
import pandas as pd
import numpy as np

# Initialize cBioPortal client
cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/v2/api-docs',
    config={"validate_requests":False,"validate_responses":False,"validate_swagger_spec": False})

# Print the operation details to see the required parameters
operation = cbioportal.Molecular_Data.fetchAllMolecularDataInMolecularProfileUsingPOST.operation
print("Required parameters:")
for param in operation.params.values():
    print(f"- {param.name}: {param.required}")

#### Get the expression data
This code defines and uses two key functions for fetching expression data from TCGA through cBioPortal's API. The `get_gene_ids()` function converts gene symbols to Entrez IDs, while `get_expression_data()` retrieves actual expression values for those genes. The code then processes this raw data into a clean format and generates visualizations showing correlations and distributions of expression levels across samples.

This creates our base functionality for getting gene expression features that we'll later use to annotate our protein network. The example runs with just three genes (TP53, BRCA1, BRCA2) to demonstrate the workflow before we apply it to all proteins in our network.

In [ ]:
from bravado.client import SwaggerClient
import pandas as pd
import numpy as np

# Initialize cBioPortal client
cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/v2/api-docs',
    config={"validate_requests":False,"validate_responses":False,"validate_swagger_spec": False})

def get_gene_ids(gene_symbols):
    genes = cbioportal.Genes.fetchGenesUsingPOST(
        geneIdType="HUGO_GENE_SYMBOL",
        geneIds=gene_symbols
    ).result()

    return {gene.hugoGeneSymbol: gene.entrezGeneId for gene in genes}

def get_expression_data(genes, study_id):
    try:
        # Get gene IDs
        gene_ids = get_gene_ids(genes)
        print(f"\nGene IDs found: {gene_ids}")

        profile_id = 'brca_tcga_rna_seq_v2_mrna'

        molecular_filter = {
            'molecularProfileIds': [profile_id],
            'entrezGeneIds': list(gene_ids.values())
        }

        # Fetch the data
        expression_data = cbioportal.Molecular_Data.fetchMolecularDataInMultipleMolecularProfilesUsingPOST(
            molecularDataMultipleStudyFilter=molecular_filter
        ).result()

        # Print sample of data
        if expression_data:
            print("\nFirst few data points:")
            for i, d in enumerate(expression_data[:5]):
                print(f"\nData point {i+1}:")
                print(f"Sample: {d.sampleId}")
                print(f"Gene: {d.entrezGeneId}")
                print(f"Value: {d.value}")

        # Create DataFrame with all data points
        raw_data = []
        for d in expression_data:
            if hasattr(d, 'value') and d.value is not None and d.value != 'NA':
                raw_data.append({
                    'sample': d.sampleId,
                    'gene': str(d.entrezGeneId),
                    'value': float(d.value)
                })

        df = pd.DataFrame(raw_data)

        # Print data shape before pivot
        print(f"\nRaw data shape: {df.shape}")
        print("\nSample counts per gene:")
        print(df.groupby('gene').size())

        # Create mapping of Entrez IDs to gene symbols
        entrez_to_symbol = {str(v): k for k, v in gene_ids.items()}
        df['gene'] = df['gene'].map(entrez_to_symbol)

        # Check for duplicates
        duplicates = df.duplicated(subset=['sample', 'gene'], keep=False)
        if duplicates.any():
            print("\nFound duplicate entries:")
            print(df[duplicates].sort_values(['sample', 'gene']))

            # Handle duplicates by taking the mean
            df = df.groupby(['sample', 'gene'])['value'].mean().reset_index()

        # Create pivot table
        pivot_df = df.pivot(index='sample', columns='gene', values='value')

        print(f"\nFinal data shape: {pivot_df.shape}")
        print("\nGenes in dataset:")
        for gene in pivot_df.columns:
            print(f"- {gene}")

        return pivot_df

    except Exception as e:
        print(f"Error fetching data: {str(e)}")
        print("\nRequest details:")
        print(f"Profile ID: {profile_id}")
        print(f"Genes: {genes}")
        print(f"Gene IDs: {gene_ids}")
        return None

# Get your known cancer genes
cancer_genes = ['TP53', 'BRCA1', 'BRCA2']

# Get expression data
expr_data = get_expression_data(cancer_genes, 'brca_tcga')

if expr_data is not None:
    # Calculate correlations
    correlations = expr_data.corr()
    print("\nExpression correlations between cancer genes:")
    print(correlations)

    # Add some basic statistics
    print("\nBasic statistics for each gene:")
    print(expr_data.describe())

    # Visualizations
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Correlation heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlations, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Gene Expression Correlations')
    plt.show()

    # Expression distributions
    plt.figure(figsize=(10, 6))
    expr_data.boxplot()
    plt.title('Expression Distribution by Gene')
    plt.ylabel('Expression Level')
    plt.xticks(rotation=45)
    plt.show()

####



In [ ]:
from bravado.client import SwaggerClient
import pandas as pd
import numpy as np

# Initialize cBioPortal client
cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/v2/api-docs',
    config={"validate_requests":False,"validate_responses":False,"validate_swagger_spec": False})

def get_gene_ids(gene_symbols):
    genes = cbioportal.Genes.fetchGenesUsingPOST(
        geneIdType="HUGO_GENE_SYMBOL",
        geneIds=gene_symbols
    ).result()
    return {gene.hugoGeneSymbol: gene.entrezGeneId for gene in genes}

def get_normalized_expression(genes, study_id):
    try:
        # Get gene IDs
        gene_ids = get_gene_ids(genes)

        # Use the z-score profile instead of raw counts
        profile_id = 'brca_tcga_rna_seq_v2_mrna_median_Zscores'

        molecular_filter = {
            'molecularProfileIds': [profile_id],
            'entrezGeneIds': list(gene_ids.values())
        }

        # Fetch the data
        expression_data = cbioportal.Molecular_Data.fetchMolecularDataInMultipleMolecularProfilesUsingPOST(
            molecularDataMultipleStudyFilter=molecular_filter
        ).result()

        # Create DataFrame
        raw_data = []
        for d in expression_data:
            if hasattr(d, 'value') and d.value is not None and d.value != 'NA':
                raw_data.append({
                    'sample': d.sampleId,
                    'gene': str(d.entrezGeneId),
                    'zscore': float(d.value)
                })

        df = pd.DataFrame(raw_data)

        # Map Entrez IDs to gene symbols
        entrez_to_symbol = {str(v): k for k, v in gene_ids.items()}
        df['gene'] = df['gene'].map(entrez_to_symbol)

        # Handle any duplicates by taking mean
        df = df.groupby(['sample', 'gene'])['zscore'].mean().reset_index()

        # Create pivot table
        pivot_df = df.pivot(index='sample', columns='gene', values='zscore')

        return pivot_df

    except Exception as e:
        print(f"Error fetching data: {str(e)}")
        return None

# Get expression data for cancer genes
cancer_genes = ['TP53', 'BRCA1', 'BRCA2']
expr_data = get_normalized_expression(cancer_genes, 'brca_tcga')

if expr_data is not None:
    # Display first few rows
    print("\nFirst 10 samples (z-scores):")
    print(expr_data.head(10))

    # Save to CSV
    expr_data.to_csv('brca_gene_expression_zscores.csv')
    print("\nFull data saved to 'brca_gene_expression_zscores.csv'")

    # Basic statistics
    print("\nSummary statistics:")
    print(expr_data.describe())

    # Number of samples with altered expression (|z-score| > 2)
    altered = (expr_data.abs() > 2).sum()
    print("\nNumber of samples with altered expression (|z-score| > 2):")
    for gene in altered.index:
        print(f"{gene}: {altered[gene]} samples")